In [ ]:
import os
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import pickle

In [ ]:
def interpolation(all_records):
    min_loss = 0.
    methods = list(all_records.keys())
    for method in methods:
        for x in all_records[method]:
            min_loss = min(min_loss, np.min(x[:, 1]))
    loss_diff = 0.01
    losses = np.arange(min_loss, 0, loss_diff)
    num_user_item_pairs = [len(all_records[methods[i]]) for i in range(len(methods))]

    num_user_item_pair = max(num_user_item_pairs)
    uplift_all = dict([(k, []) for k in methods])

    for i in range(num_user_item_pair):
        for method in methods:
            record_idx = 0
            print(f'Interpolating {method} {i}')
            uplift, min_loss = all_records[method][i][0]
            uplifts = np.zeros(len(losses))
            for j in range(len(losses)):
                if record_idx == 0:
                    if losses[j] <= min_loss:
                        uplifts[j] = uplift
                    else:
                        while losses[j] > all_records[method][i][record_idx][1]:
                            record_idx += 1
                        uplift_left = uplift
                        uplift, min_loss = all_records[method][i][record_idx]
                        uplifts[j] = uplift_left
                else:
                    if losses[j] <= min_loss:
                        uplifts[j] = uplift_left
                    else:
                        while losses[j] > all_records[method][i][record_idx][1]:
                            record_idx += 1
                        uplift_left = uplift
                        uplift, min_loss = all_records[method][i][record_idx]
                        uplifts[j] = uplift_left
            uplifts[-1], _ = all_records[method][i][-1]
            uplift_all[method].append(uplifts)
    return losses, uplift_all

In [ ]:
ciao_data_dir = ['./metric/group_user_var2_ph_seed0/groundtruth/1.0_0_1000_1_1/ciao100.10.20.70.250.251.010.00.5/', './metric/group_user_var2_ph_seed0/groundtruth/1.0_0_1000_1_50/ciao100.10.20.70.250.251.010.00.5/', './metric/group_user_var2_ph_seed0/groundtruth/1.0_0_1000_1_75/ciao100.10.20.70.250.251.010.00.5/', './metric/group_user_var2_ph_seed0/groundtruth/1.0_0_1000_1_100/ciao100.10.20.70.250.251.010.00.5/']

methods = ['spill_groundtruth', 'spill_diffnet', 'spill_mf', 'spill_lightgcn', 'spill_simulation_lightgcn_scale_revision', 'spill_simulation_mf_scale_revision', 'spill_simulation_diffnet_scale_revision', 'spill_gatv2_spill_var2']

In [ ]:
ciao_losses, ciao_uplifts = [], []

for dir in ciao_data_dir:
    # methods = os.listdir(os.path.join(dir))
    # methods = [method for method in methods if method not in exclude]
    all_records = dict([(k, []) for k in methods])
    for method in methods:
        user_item_records = sorted(os.listdir(os.path.join(dir, method)))
        for user_item_record in user_item_records:
            if 'metric' in user_item_record:
                file_dir = os.path.join(dir, method, user_item_record)
                with open(file_dir, 'r') as f:
                    lines = f.readlines()
                    lines = [line.split(' ') for line in lines]
                    lines = [[float(x) for x in line] for line in lines]
                    lines = np.array(lines)
                    all_records[method].append(lines[:, [2, 5]])
    losses, uplift_all = interpolation(all_records)
    ciao_losses.append(losses)
    ciao_uplifts.append(uplift_all)

In [ ]:
ciao_uplifts_save = []

for i in range(4):
    ciao_i = {}
    for method in methods:
        ciao_i[method] = np.mean(ciao_uplifts[i][method], axis=0)

    ciao_uplifts_save.append(ciao_i)


In [ ]:
ciao_uplifts_save

In [ ]:

with open('plot_datas/ciao_losses.pkl', 'wb') as file:
    pickle.dump(ciao_losses, file)

with open('plot_datas/ciao_uplifts.pkl', 'wb') as file:
    pickle.dump(ciao_uplifts_save, file)
